In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')
df_test = df_test.drop(['id'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x = df_train.drop('price_range',axis=1)
y = df_train['price_range']

scaler.fit(x)
x_transformed = scaler.transform(x)

x_train,x_test,y_train,y_test = train_test_split(x_transformed,y,test_size=0.3, random_state=42)

In [ ]:
#SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix,classification_report
model1 = SVC(kernel = 'rbf', gamma = 'scale', random_state=20)
model2 = SVC(kernel = 'rbf', gamma = 'auto', random_state=20)
model3 = SVC(kernel = 'linear', random_state=20)
model4 = SVC(kernel = 'poly', random_state=20)
model5 = SVC(kernel = 'poly', degree = 5, random_state=20)

model5.fit(x_train,y_train)
y_train_pred = model5.predict(x_train)
y_test_pred = model5.predict(x_test)

print("Train Set Accuracy:"+str(accuracy_score(y_train_pred,y_train)*100))
print("Test Set Accuracy:"+str(accuracy_score(y_test_pred,y_test)*100))

output: <br>
RBF kernel <br>
gamma = auto<br>
train = 98.785, test = 86.833 <br>
gamma = scale<br>
train = 98.785, test = 86.833 <br>
linear kernel <br>
train = 97.714, test = 95.333 <br>
poly kernel<br>
train = 97.142, test = 78.5<br>
poly kernel degree=5<br>
train = 94.928, test = 54.0<br>

In [ ]:
model = SVC(C = 0.005, kernel = 'linear', random_state=20)
model.fit(x_train,y_train)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

print("Train Set Accuracy:"+str(accuracy_score(y_train_pred,y_train)*100))
print("Test Set Accuracy:"+str(accuracy_score(y_test_pred,y_test)*100))

output:  <br>
c = 0.005 -> train = 87.5 , test = 85.666 <br>
c = 0.2 -> train = 95.785, test = 93.666 <br>
c = 0.8 -> train = 97.357 , test = 95.833 <br>
c = 2 -> train = 98.0 , test = 95.833 <br>
c = 8 -> train = 98.285 , test = 96.5 <br>
c = 50 -> train = 98.5 , test = 97.666

# Question 5

In [ ]:
df_train['sc_area'] = df_train['sc_h'] * df_train['sc_w']
df_test['sc_area'] = df_test['sc_h'] * df_test['sc_w']

In [ ]:
df_train['battery_power'].describe()
# value_counts

In [ ]:
'''equal frequency binning'''
labels = ['very_low', 'low', 'medium', 'high', 'very_high']
df_train['bin'] = pd.qcut(df_train['battery_power'], q=5, precision=1, labels=labels)

'''equal size binning'''
# bins = np.linspace(df_train['battery_power'].min(),df_train['battery_power'].max(),6)
# df_train['bin'] = pd.cut(df_train['battery_power'], bins=bins, labels=labels, include_lowest=True)

'''manual size binning'''
# man_bins = [501, 670, 1020, 1390, 1790, 1998]
# df_train['bin'] = pd.cut(df_train['battery_power'], bins=man_bins, labels=labels, include_lowest=True)
# df_train['bin'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_train['bin'], bins=5)
plt.show()

In [ ]:
# pd.get_dummies(df_train['bin'])

# این پایینی غلطهههههه <br> هر وکتور رو 2000 * 5 میگیره و با >اپلای< هم ارور میداد

In [ ]:
from sklearn import preprocessing

labelEnc = preprocessing.LabelEncoder()
m = labelEnc.fit_transform(df_train['bin'])
enc = preprocessing.OneHotEncoder()
enc.fit(m.reshape(-1, 1))
df_train['binvec'] = enc.transform(m.reshape(-1, 1))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth =7, min_samples_split =2, random_state = 0)
clf.fit(x_train,y_train)
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

print("Train Set Accuracy:"+str(accuracy_score(y_train_pred,y_train)*100))
print("Test Set Accuracy:"+str(accuracy_score(y_test_pred,y_test)*100))

max_depth , min sample, train score, test score: <br>
(3, 2, 77.9, 75.33) <br>
(7, 2, 96.21, 82.5)<br>
(7, 10, 94.78, 82.3) <br>
(7, 100, 84.5, 74.3) <br>
omgh ziad -> deqqat ziad (but causes overfitting) <br>
minimum sample ziad -> deghat kamtar but hassasiat be noise ham kamtar

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(max_depth =7, min_samples_split =100, random_state = 0)
rnd_clf.fit(x_train,y_train)
y_train_pred = rnd_clf.predict(x_train)
y_test_pred = rnd_clf.predict(x_test)

print("Train Set Accuracy:"+str(accuracy_score(y_train_pred,y_train)*100))
print("Test Set Accuracy:"+str(accuracy_score(y_test_pred,y_test)*100))

max_depth , min sample, train score, test score: <br>
(3, 2, 83.14, 79.5) <br>
(7, 2, 98.35, 85.33)<br>
(7, 10, 96.92, 86.16) <br>
(7, 100, 84.5, 74.3) <br>
random forest -> accuracy ziad, exe time ziad, overfit nemishe chon kinda average migire from all the random trees in the forest. also hamin dalile exec time ziadeshe.